# Script para automatização de processos de consulta.
 -Inserindo um estado ele procisa me retornar:
     --Numero total de aeroportos com suas classificações
     --Tipos de pavimento de aeroportos e quantidade
     --Total de aeroportos e sua categoria
     --extensão total de rodovias de acordo com seus pavimentos

In [12]:
# Importando bibliotecas necessárias
import geopandas as gpd
import os
import matplotlib
import matplotlib.pyplot as plt
import pygeos

def path_lista():
    path_municipios = os.getcwd()+'//dados//municipios.shp'
    path_aeroportos = os.getcwd()+'//dados//aeroportos.shp'
    path_rodovias = os.getcwd()+'//dados//rodovias.shp'
    
    return[path_municipios, path_aeroportos, path_rodovias]

def open_shapefile():
    lista_shapefile = path_lista()

    shapefiles=[]
    for i in lista_shapefile:
        shapefiles.append(gpd.read_file(i))
        
    return shapefiles    

In [14]:
open_shapefile()

[        id  id_pk geocodigo           municipio  uf        regiao  area  \
 0        1      1   5100102            Acorizal  MT  Centro-Oeste  None   
 1        2    104   5107156  Reserva Do Cabaçal  MT  Centro-Oeste  None   
 2        3    144   5200100           Abadiânia  GO  Centro-Oeste  None   
 3        4    147   5200175  Água Fria De Goiás  GO  Centro-Oeste  None   
 4        5    160   5201306             Anicuns  GO  Centro-Oeste  None   
 ...    ...    ...       ...                 ...  ..           ...   ...   
 6124  6125   6120   4128401                Uraí  PR           Sul  None   
 6125  6126   6121   4128500      Wenceslau Braz  PR           Sul  None   
 6126  6127   6123   4128559  Vera Cruz Do Oeste  PR           Sul  None   
 6127  6128   6125   4128625        Alto Paraíso  PR           Sul  None   
 6128  6129   6128   4128708            Vitorino  PR           Sul  None   
 
                                                geometry  
 0     POLYGON ((-56.28986 